# Исследование алгоритмов

### Проверка и установка рабочей директории, должен быть корень проекта

In [1]:
%pwd

'C:\\Users\\Kuroha\\source\\repos_py\\bauman_final_project\\notebooks'

In [2]:
%cd ..

C:\Users\Kuroha\source\repos_py\bauman_final_project


### Загрузка датасета:

In [3]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from src.utils import *

In [4]:
path = get_filepath(DATA_PROCESSED_TRAIN, is_raw=False)
df = pd.read_csv(path, parse_dates=['date'])

In [5]:
df.head().T

,0,1,2,3,4
date,2008-01-01 00:00:00,2008-01-02 00:00:00,2008-01-03 00:00:00,2008-01-04 00:00:00,2008-01-05 00:00:00
latitude,0.312439,0.312439,0.312439,0.312439,0.312439
longitude,0.699807,0.699807,0.699807,0.699807,0.699807
temperature,0.438202,0.314607,0.146067,0.168539,0.325843
pressure,0.414474,0.519737,0.546053,0.546053,0.361842
cloud,1.0,0.0,0.0,0.666667,1.0
wind_spd,0.333333,0.166667,0.0,0.0,0.666667
is_fallback_data,1,1,1,1,1
water_level,138.0,138.0,138.0,138.0,138.0
uid_0,1.0,1.0,1.0,1.0,1.0


### Подготовка данных

Необходимо проверить, зависит ли качество моделей от способа кодирования погоды: одной колонкой с наличием осадков, двумя - дождь и снег, либо тремя - дождь, гроза и снег.

Датасеты с разные вариантами кодирования погоды будут именоваться как **v1**, **v2**, **v3**.

In [6]:
df.columns

Index(['date', 'latitude', 'longitude', 'temperature', 'pressure', 'cloud',
       'wind_spd', 'is_fallback_data', 'water_level', 'uid_0', 'uid_1',
       'uid_2', 'uid_3', 'uid_4', 'uid_5', 'uid_6', 'uid_7', 'uid_8', 'uid_9',
       'uid_10', 'uid_11', 'uid_12', 'uid_13', 'uid_14', 'year', 'day_sin',
       'day_cos', 'weather_v1_precip', 'weather_v2_rain', 'weather_snow',
       'weather_v3_rain', 'weather_v3_storm', 'north', 'south', 'west', 'east',
       'pressure_upper', 'pressure_lower', 'wind_spd_upper'],
      dtype='object')

Для обучения модели используем данные за 2008-2016 года, а для тестирования - за 2017.

In [7]:
df_X_train = df.loc[(df['date'] < '2017-01-01')]
df_X_test = df.loc[(df['date'] >= '2017-01-01')]
y_train = df_X_train['water_level']
y_test = df_X_test['water_level']

print(f'''Размерность оригинального датасета: {df.shape}
Размерность тренировочного датасета: {df_X_train.shape}
Размерность тестового датасета: {df_X_test.shape}
Размер тестовой выборки: {df_X_test.shape[0] / df_X_train.shape[0] * 100:.2f}%''')

Размерность оригинального датасета: (71159, 39)
Размерность тренировочного датасета: (61535, 39)
Размерность тестового датасета: (9624, 39)
Размер тестовой выборки: 15.64%


In [8]:
def get_all_df(input_df):
    df_all_base = input_df.drop(['date', 'water_level'], axis=1)
    df_all_v1 = df_all_base.drop(['weather_v2_rain', 'weather_snow', 'weather_v3_rain', 'weather_v3_storm'], axis=1)
    df_all_v2 = df_all_base.drop(['weather_v1_precip', 'weather_v3_rain', 'weather_v3_storm'], axis=1)
    df_all_v3 = df_all_base.drop(['weather_v1_precip', 'weather_v2_rain'], axis=1)
    return [df_all_v1, df_all_v2, df_all_v3]

df_X_train_all = get_all_df(df_X_train)
df_X_test_all = get_all_df(df_X_test)